In [1]:
import utils
inp = utils.get_input(2019, 13)[:-1]
print(inp[:96])

Found cached input for 2019 day 13
1,380,379,385,1008,2979,673982,381,1005,381,12,99,109,2980,1102,0,1,383,1101,0,0,382,20102,1,382


## machine

In [2]:
import operator
from functools import partial
from itertools import repeat, chain
from collections import namedtuple, defaultdict


class IntCode:
    Op = namedtuple('Op', ['func', 'params', 'resolve_last_as_ptr'])

    def __init__(self, program, pointer=0, rel_base=0, **kwargs):
        self._mem = defaultdict(int, enumerate(program))
        self.ptr = pointer
        self.rel = rel_base
        self.state = "started"
        self.output = None
        self.inputs = []
        self._kwargs = kwargs
        self._ops = {
            1: IntCode.Op(partial(IntCode._math, operator.add), 3, True),
            2: IntCode.Op(partial(IntCode._math, operator.mul), 3, True),
            3: IntCode.Op(IntCode._inp, 1, True),
            4: IntCode.Op(IntCode._out, 1, False),
            5: IntCode.Op(partial(IntCode._jump, lambda x: x != 0), 2, False),
            6: IntCode.Op(partial(IntCode._jump, lambda x: x == 0), 2, False),
            7: IntCode.Op(partial(IntCode._math, operator.lt), 3, True),
            8: IntCode.Op(partial(IntCode._math, operator.eq), 3, True),
            9: IntCode.Op(IntCode._base, 1, False),
            99: IntCode.Op(IntCode._halt, 0, False),
        }

    @property
    def mem(self):
        return list(self._mem.values())

    def run(self, inputs=None):
        self.inputs.extend(inputs or [])

        while self.state != "halted":
            instruction = self._mem[self.ptr]
            opcode, modes = self._opcode(instruction), self._modes(instruction)
            op = self._ops[opcode]
            params_raw = [self._mem[i] for i in range(self.ptr + 1, self.ptr + op.params + 1)]
            params = self.resolve(params_raw, modes, op.resolve_last_as_ptr)

            orig_ptr = self.ptr
            self._mem, self.ptr, self.rel, output, self.state = \
                op.func(self._mem, self.ptr, self.rel, *params, inputs=self.inputs, **self._kwargs)
            self.output = output if output is not None else self.output

            if self._kwargs.get('_debug', False):
                print(f"{instruction},{','.join(map(str, params_raw))}",
                      f"-> {opcode:2d} {modes} {params}",
                      f"-> {output, self.ptr, self.rel}")

            if (output is not None) or (self.state == "blocked"):
                yield self

            if self.ptr == orig_ptr:
                self.ptr = self.ptr + op.params + 1
        yield self

    def resolve(self, params, modes, resolve_last_as_ptr):
        def _resolve(p, m, as_ptr):
            if int(m) == 0:
                return self._mem[p] if not as_ptr else p
            elif int(m) == 2:
                return self._mem[self.rel + p] if not as_ptr else self.rel + p
            return p
        resolve_as_ptr = chain(repeat(False, len(params) - 1), [resolve_last_as_ptr])
        return list(map(lambda t: _resolve(*t), zip(params, modes, resolve_as_ptr)))

    @staticmethod
    def _opcode(instruction):
        return int(str(instruction)[-2:])

    @staticmethod
    def _modes(instruction):
        return f"{instruction:05d}"[:3][::-1]
    
    # operations
    
    @staticmethod
    def _math(func, mem, ptr, rel, a, b, out, **kwargs):
        mem[out] = int(func(a, b))
        return mem, ptr, rel, None, "running"

    @staticmethod
    def _inp(mem, ptr, rel, out, inputs, **kwargs):
        try:
            mem[out] = int(inputs.pop(0))
            return mem, ptr, rel, None, "running"
        except StopIteration:
            return mem, ptr, rel, None, "blocked"

    @staticmethod
    def _out(mem, ptr, rel, val, **kwargs):
        return mem, ptr, rel, val, "running"

    @staticmethod
    def _jump(func, mem, ptr, rel, cond, val, **kwargs):
        return mem, (val if func(cond) else ptr), rel, None, "running"

    @staticmethod
    def _base(mem, ptr, rel, val, **kwargs):
        return mem, ptr, (rel + val), None, "running"
    
    @staticmethod
    def _halt(mem, ptr, rel, *args, **kwargs):
        return mem, ptr, rel, None, "halted"


In [3]:
from IPython.display import clear_output

program = list(map(int, inp.split(',')))
program[0] = 2

grid = {}

vm = IntCode(program, _debug=False)
vm_gen = vm.run()


def viz(grid):
    grid_ = []
    max_x = max(k[0] for k, v in grid.items()) + 1
    max_y = max(k[1] for k, v in grid.items()) + 1
    for y in range(max_y):
        row = [''] * max_x
        for x in range(max_x):
            if (x, y) in grid:
                row[x] = grid[(x, y)]
        grid_.append(row)
    return '\n'.join(map(lambda x: ''.join(map(str, x)), grid_)).replace('0', ' ')

ball_x, paddle_x, score = 0, 0, 0

while True:
    try:
        x, y, vm = next(vm_gen).output, next(vm_gen).output, next(vm_gen)
        z = vm.output

        if (x, y) == (-1, 0):
            score = z
        else:
            grid[(x, y)] = z

        if z == 3:
            paddle_x = x
        elif z == 4:
            ball_x = x
        # overwrite inputs
        vm.inputs = [-1] if ball_x < paddle_x else [1] if ball_x > paddle_x else [0]
        
        if (3 in grid.values()) and (4 in grid.values()):
            clear_output(wait=True)
            print(viz(grid))
            print(f"score: {score:6,d}")

    except StopIteration:
        break


# print(score)

111111111111111111111111111111111111111111111
1                                           1
1                                           1
1                                           1
1                                           1
1                                           1
1                                           1
1                                           1
1          4                                1
1                                           1
1                                           1
1                                           1
1                                           1
1                                           1
1                                           1
1                                           1
1                                           1
1                                           1
1                                           1
1                                           1
1                                           1
1                                 